In [1]:
from Bio.PDB.PDBParser import PDBParser
import argparse
from f_parse_pdb_general import parse_pdb
import numpy as np
from rdkit import Chem
import os
from tqdm import tqdm
import pickle


def parse_sdf_file(file_path):
    suppl = Chem.SDMolSupplier(file_path, sanitize = True, removeHs=True, strictParsing=True)
    molecules = []
    for mol in suppl:
        if mol is not None:
            molecules.append(mol)
    return molecules

def load_object(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

##################
# def arg_parser():
#     parser = argparse.ArgumentParser(description='Preprocess PDBbind data for DTI5')
#     parser.add_argument('--data_dir', type=str, required=True, help='Path to the data directory containing all proteins(PDB) and ligands (SDF)')
#     parser.add_argument('--affinity_dict', type=str, required=True, help='Path to the affinity dictionary')
#     return parser.parse_args()

# args = arg_parser()
# data_dir = args.data_dir
# affinity_dict_path = args.affinity_dict
##################

data_dir = '/cluster/work/math/dagraber/DTI/PDBbind1'
affinity_dict_path = '/cluster/work/math/dagraber/DTI/PDBbind_affinity_dict.pkl'

affinity_dict = load_object(affinity_dict_path)


# Get sorted lists of proteins and ligands (dirEntry objects) in the data_dir
proteins = sorted([protein for protein in os.scandir(data_dir) if protein.name.endswith('protein.pdb')], key=lambda x: x.name)
ligands = sorted([ligand for ligand in os.scandir(data_dir) if ligand.name.endswith('ligand.sdf')], key=lambda x: x.name)

if not len(proteins) == len(ligands):
    raise ValueError('Number of proteins and ligands does not match')
else:
    print(f'Number of Protein PDBs: {len(proteins)}')
    print(f'Number of Ligand SDFs: {len(ligands)}')


Number of Protein PDBs: 16
Number of Ligand SDFs: 16


In [2]:
# Initialize Log File:
log_folder = os.path.join(data_dir,'.logs/')
if not os.path.exists(log_folder): os.makedirs(log_folder)
log_file_path = os.path.join(log_folder, "preprocessing_logs.txt")
log = open(log_file_path, 'a')
log.write("Data Preprocessing - Log File:\n")
log.write("\n")

1

In [3]:
# Initialize PDB Parser
parser = PDBParser(PERMISSIVE=1, QUIET=True)

amino_acids = ["ALA","ARG","ASN","ASP","CYS","GLN","GLU","GLY","HIS","ILE","LEU","LYS","MET","PHE","PRO","SER","THR","TRP","TYR","VAL"]
known_hetatms = ['ZN','MG','NA','MN','CA','K','NI','FE','CO','HG','CD','CU','CS','AU','LI','GA','IN','BA','RB','SR']
known_residues = amino_acids + known_hetatms


# Here start a loop over the complexes
#----------------------------------------------------------
for protein, ligand in zip(proteins, ligands):
    
    if protein.name.split('_')[0] != ligand.name.split('_')[0]:
        raise ValueError('Protein and Ligand do not match')
    else:
        protein_id = protein.name.split('_')[0]
        protein_path = protein.path
        ligand_path = ligand.path
    
    log_string = f'{protein_id}: '
    
    # -----------------------------------------------------
    # PRESELECTION OF COMPLEXES
    # -----------------------------------------------------

    # TEST 1: AFFINITY DATA - Continue only if there is a valid affinity value available for this complex
    if protein_id not in affinity_dict.keys():
        log_string += 'Protein is not processed: No valid affinity value'
        log.write(log_string + "\n")
        continue

    # TEST 2: LIGAND PARSING - Continue only if the parsed ligand has been processed successfully, else skip this complex
    ligand = parse_sdf_file(ligand_path)
    if len(ligand) == 1: mol = ligand[0]
    else:
        log_string += 'Ligand could not be parsed successfully'
        log.write(log_string + "\n")
        continue
    
    # TEST 3: LIGAND SIZE - Get coordinate Matrix of the ligand molecule - Continue only if the ligand has at least 6 heavy atoms
    conformer = mol.GetConformer()
    coordinates = conformer.GetPositions()
    pos = np.array(coordinates)
    if pos.shape[0]<5:
        log_string += 'Ligand is smaller than 5 Atoms and is therefore skipped'
        log.write(log_string + "\n")
        continue


    # PARSING OF PROTEIN PDB TO GENERATE COORDINATE MATRIX
    # -----------------------------------------------------

    with open(protein_path) as pdbfile:
        protein = parse_pdb(parser, protein_id, pdbfile)


    protein_atomcoords = np.array([], dtype=np.int64).reshape(0,3)
    res_list = []
    residue_memberships = []
        
    clean_aa_chain = False
    chain_too_long = False
    residue_idx = 1

    # Iterate over the chains in the protein
    for chain in protein:

        chain_comp = protein[chain]['composition']

        # CHAIN CONTAINS ONLY AMINO ACIDS (AND HETATMS in chain)
        if chain_comp == [True, False] or chain_comp == [True, True]:
            clean_aa_chain = True

            # If the chain is longer than 1024, skip the complex
            if len(protein[chain]['aa_seq']) > 1022:
                chain_too_long = True
                break
            
            for residue in protein[chain]['aa_residues']:
                res_dict = protein[chain]['aa_residues'][residue]

                # Append the coords of the residue to the protein_atomcoords
                protein_atomcoords = np.vstack((protein_atomcoords, res_dict['coords']))

                res_list.append((residue_idx, res_dict['resname']))
                
                memb = [residue_idx for atom in res_dict['atom_indeces']]
                residue_memberships.extend(memb)

                residue_idx += 1
        
        # CHAIN CONTAINS HETATMS BUT NO AMINO ACIDS
        elif protein[chain]['composition'] == [False, True]: 
            
            for hetatm_res in protein[chain]['hetatm_residues']:
                hetatmres_dict = protein[chain]['hetatm_residues'][hetatm_res]

                # Append the coords of the residue to the protein_atomcoords
                protein_atomcoords = np.vstack((protein_atomcoords, hetatmres_dict['hetatmcoords']))

                res_list.append((residue_idx, hetatmres_dict['resname']))

                memb = [residue_idx for atom in hetatmres_dict['atoms']]
                residue_memberships.extend(memb)

                residue_idx +=1
        
        # To do --> If we remove all HOH, we can skip this code
        # CHAIN CONTAINS ONLY WATER MOLECULES        
        elif protein[chain]['composition'] == [False, False]:
            pass



    # If a chain is too long to generate an ESM Embedding, skip the complex
    if chain_too_long:
        log_string += 'Protein AA sequence too long for ESM'
        log.write(log_string + "\n")
        continue




    # COMPUTE CONNECTIVITY BETWEEN LIGAND AND PROTEIN ATOMS
    # (ONLY BASED ON DISTANCE, NOT ON ACCESSIBILITY)
    # -----------------------------------------------------

    if not clean_aa_chain:
        log_string += 'No clean AA_chain has been found, complex is skipped'
        log.write(log_string + "\n")
        continue


    # Prepare data for creating connections with f_connect_to_accessible_aa.py
    residue_membership = np.array(residue_memberships)

    # Make a preselection of the enzyme atoms, only include those that are closer than 4A from any of the substrate atoms
    max_len = 4
    diff = protein_atomcoords[np.newaxis, :, :] - pos[:, np.newaxis, :]
    pairwise_distances = np.linalg.norm(diff, axis=2)
    close = pairwise_distances <= max_len + 1

    connections = [np.unique(residue_membership[np.where(row)]) for row in close]
    connected_res_num = sorted(list(set([atm for l in connections for atm in l])))
    connected_res_name = [res_list[aa-1][1] for aa in connected_res_num]


    unknown_res = [(res not in known_residues and res.strip('0123456789') not in known_residues) for res in connected_res_name]
    if any(unknown_res):
        log_string += 'Ligand has been connected to a unknown protein residue, the complex is therefore skipped'
        log.write(log_string + "\n")
        continue

    
    # EXPORT DATA
    # -------------------------------------------------------

    # Export protein dictionary as pkl
    filepath = os.path.join(data_dir, f'{protein_id}_protein_dict.pkl')
    with open(filepath, 'wb') as fp:
        pickle.dump(protein, fp)

    # Export CONNECTIONS as dict
    connections_dict = {'connections':connections, 'res_num':connected_res_num, 'res_name':connected_res_name}
    filepath = os.path.join(data_dir, f'{protein_id}_connections.pkl')
    with open(filepath, 'wb') as fp:
        pickle.dump(connections_dict, fp)


    log_string += 'Successful'
    log.write(log_string + "\n")

log.close()

[15:35:51] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[15:35:52] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[15:35:52] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[15:35:52] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[15:35:52] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[15:35:53] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[15:35:53] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[15:35:53] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[15:35:54] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol